In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,28411
2,Huelva,Confirmados PDIA 14 días,1143
3,Huelva,Tasa PDIA 14 días,"222,73320731921197"
4,Huelva,Confirmados PDIA 7 días,485
5,Huelva,Total Confirmados,28586
6,Huelva,Curados,24116
7,Huelva,Fallecidos,364


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  28411.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7369.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1169 personas en los últimos 7 días 

Un positivo PCR cada 547 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,28411.0,485.0,1143.0,513170.0,94.510591,222.733207,44.0
Huelva-Costa,16585.0,201.0,428.0,289548.0,69.418542,147.816597,29.0
Condado-Campiña,8917.0,239.0,593.0,156231.0,152.978602,379.566155,27.0
Huelva (capital),7369.0,123.0,263.0,143837.0,85.513463,182.845860,20.0
Lucena del Puerto,230.0,30.0,42.0,3261.0,919.963201,1287.948482,6.0
Sierra de Huelva-Andévalo Central,2535.0,39.0,105.0,67391.0,57.871229,155.807155,4.0
Rociana del Condado,507.0,12.0,46.0,7939.0,151.152538,579.418063,4.0
Almonte,1493.0,71.0,181.0,24507.0,289.713143,738.564492,4.0
Ayamonte,1277.0,12.0,14.0,21104.0,56.861259,66.338135,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cumbres Mayores,49.0,5.0,24.0,1749.0,285.877644,1372.212693,0.0
Cumbres de San Bartolomé,10.0,2.0,5.0,376.0,531.914894,1329.787234,0.0
Lucena del Puerto,230.0,30.0,42.0,3261.0,919.963201,1287.948482,6.0
Cortelazor,7.0,2.0,3.0,299.0,668.896321,1003.344482,0.0
Galaroza,27.0,3.0,11.0,1382.0,217.076700,795.947902,0.0
Almonte,1493.0,71.0,181.0,24507.0,289.713143,738.564492,4.0
Rociana del Condado,507.0,12.0,46.0,7939.0,151.152538,579.418063,4.0
Beas,197.0,14.0,24.0,4341.0,322.506335,552.868003,3.0
Bonares,253.0,13.0,29.0,6060.0,214.521452,478.547855,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Palma del Condado (La),1030.0,2.0,17.0,10801.0,18.516804,157.392834,0.0,0.117647
Aljaraque,992.0,2.0,13.0,21474.0,9.313589,60.538325,0.0,0.153846
Niebla,193.0,1.0,6.0,4116.0,24.295432,145.772595,0.0,0.166667
Aracena,507.0,2.0,10.0,8255.0,24.227741,121.138704,0.0,0.200000
Cumbres Mayores,49.0,5.0,24.0,1749.0,285.877644,1372.212693,0.0,0.208333
Gibraleón,582.0,5.0,20.0,12737.0,39.255712,157.022847,0.0,0.250000
Rociana del Condado,507.0,12.0,46.0,7939.0,151.152538,579.418063,4.0,0.260870
Galaroza,27.0,3.0,11.0,1382.0,217.076700,795.947902,0.0,0.272727
Almonaster la Real,65.0,2.0,7.0,1788.0,111.856823,391.498881,0.0,0.285714
